In [1]:
import pandas as pd
import numpy as np

df = (pd.read_csv(r"C:\Users\matte\OneDrive\Desktop\GitHub\data\causal\online_mkt.csv")
              .astype({"date":"datetime64[ns]"}))

df.head()

,app_download,population,city,state,date,post,treated
0,3066.0,12396372,sao_paulo,sao_paulo,2022-03-01,0,1
1,2701.0,12396372,sao_paulo,sao_paulo,2022-03-02,0,1
2,1927.0,12396372,sao_paulo,sao_paulo,2022-03-03,0,1
3,1451.0,12396372,sao_paulo,sao_paulo,2022-03-04,0,1
4,1248.0,12396372,sao_paulo,sao_paulo,2022-03-05,0,1


In [2]:
treated = list(df.query("treated==1")["city"].unique())
treated

['sao_paulo', 'porto_alegre', 'joao_pessoa']

In [3]:
df_norm = df.assign(
            app_download_pct = 100*df["app_download"]/df["population"]
        )

df_norm.head()

,app_download,population,city,state,date,post,treated,app_download_pct
0,3066.0,12396372,sao_paulo,sao_paulo,2022-03-01,0,1,0.024733
1,2701.0,12396372,sao_paulo,sao_paulo,2022-03-02,0,1,0.021789
2,1927.0,12396372,sao_paulo,sao_paulo,2022-03-03,0,1,0.015545
3,1451.0,12396372,sao_paulo,sao_paulo,2022-03-04,0,1,0.011705
4,1248.0,12396372,sao_paulo,sao_paulo,2022-03-05,0,1,0.010067


In [4]:
tr_period = df_norm.query("post==1")["date"].min()
tr_period

Timestamp('2022-05-01 00:00:00')

In [7]:
def reshape_sc_data(df,
                            geo_col, 
                            time_col,
                            y_col,
                            tr_geos,
                            tr_start):
            
            df_pivot = df.pivot(index=time_col, columns=geo_col, values=y_col)
            
            y_co = df_pivot.drop(columns=tr_geos)
            y_tr = df_pivot[tr_geos]
            
            y_pre_co = y_co[df_pivot.index < tr_start]
            y_pre_tr = y_tr[df_pivot.index < tr_start]
            
            y_post_co = y_co[df_pivot.index >= tr_start]
            y_post_tr = y_tr[df_pivot.index >= tr_start]
            
            return y_pre_co, y_pre_tr, y_post_co, y_post_tr

In [8]:
y_pre_co, y_pre_tr, y_post_co, y_post_tr = reshape_sc_data(
            df_norm,
            geo_col="city",
            time_col="date",
            y_col="app_download_pct",
            tr_geos=treated,
            tr_start=str(tr_period)
        )

y_pre_tr.head()

city,sao_paulo,porto_alegre,joao_pessoa
date,,,
2022-03-01,0.024733,0.004288,0.022039
2022-03-02,0.021789,0.008107,0.020344
2022-03-03,0.015545,0.004891,0.012352
2022-03-04,0.011705,0.002948,0.018285
2022-03-05,0.010067,0.006767,0.000000


In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(y_pre_co, y_pre_tr.mean(axis=1))

# extract the weights
weights_lr = model.coef_
weights_lr.round(3)

array([-0.65 , -0.058, -0.239,  0.971,  0.03 , -0.204,  0.007,  0.095,
        0.102,  0.106,  0.074,  0.079,  0.032, -0.5  , -0.041, -0.154,
       -0.014,  0.132,  0.115,  0.094,  0.151, -0.058, -0.353,  0.049,
       -0.476, -0.11 ,  0.158, -0.002,  0.036, -0.129, -0.066,  0.024,
       -0.047,  0.089, -0.057,  0.429,  0.23 , -0.086,  0.098,  0.351,
       -0.128,  0.128, -0.205,  0.088,  0.147,  0.555,  0.229])

In [13]:
y0_tr_hat = y_post_co.dot(weights_lr)

In [14]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import (check_X_y, check_array,
                                               check_is_fitted)
import cvxpy as cp

class SyntheticControl(BaseEstimator, RegressorMixin):

             def __init__(self,):
                 pass

             def fit(self, y_pre_co, y_pre_tr):

                 y_pre_co, y_pre_tr = check_X_y(y_pre_co, y_pre_tr)
             
                 w = cp.Variable(y_pre_co.shape[1])
                 
                 objective = cp.Minimize(cp.sum_squares(y_pre_co@w - y_pre_tr))
                 constraints = [cp.sum(w) == 1, w >= 0]
                 
                 problem = cp.Problem(objective, constraints)
                 
                 self.loss_ = problem.solve(verbose=False)
                 self.w_ = w.value
                 
                 self.is_fitted_ = True
                 return self
                 
                 
             def predict(self, y_co):

                 check_is_fitted(self)
                 y_co = check_array(y_co)
                 
                 return y_co @ self.w_

(CVXPY) Mar 21 06:54:14 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: Impossibile trovare il modulo specificato.')


In [16]:
model = SyntheticControl()
model.fit(y_pre_co, y_pre_tr.mean(axis=1))

         # extrac the weights
model.w_.round(3)

c:\Users\matte\anaconda3\envs\py_env\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.084, 0.039, 0.085,
       0.003, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.062, 0.121,
       0.   , 0.072, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.095, 0.   , 0.   , 0.   , 0.022,
       0.116, 0.   , 0.068, 0.046, 0.   , 0.   , 0.   , 0.088, 0.   ,
       0.098, 0.   ])

In [18]:
df_norm_cov = (pd.read_csv(r"C:\Users\matte\OneDrive\Desktop\GitHub\data\causal\online_mkt_cov.csv")
                        .astype({"date":"datetime64[ns]"}))

df_norm_cov.head()

,app_download,population,city,state,date,post,treated,app_download_pct,comp_download_pct
0,3066.0,12396372,sao_paulo,sao_paulo,2022-03-01,0,1,0.024733,0.026280
1,2701.0,12396372,sao_paulo,sao_paulo,2022-03-02,0,1,0.021789,0.023925
2,1927.0,12396372,sao_paulo,sao_paulo,2022-03-03,0,1,0.015545,0.018930
3,1451.0,12396372,sao_paulo,sao_paulo,2022-03-04,0,1,0.011705,0.015858
4,1248.0,12396372,sao_paulo,sao_paulo,2022-03-05,0,1,0.010067,0.014548
